In [71]:
from ultralytics import YOLO
import cv2

# Cargar el modelo preentrenado (por ejemplo, YOLOv8n, YOLOv8s, etc.)
model = YOLO("yolov8n.pt") 

In [72]:
# Cargar imagen
image_path = "example.jpg"
results = model(image_path)
print(results)  # Imprimir resultados


image 1/1 /home/dani/Desktop/futbol/example.jpg: 416x640 5 persons, 3.5ms
Speed: 1.1ms preprocess, 3.5ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana',

In [73]:
# Filtrar personas y mostrar resultados
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls_id = int(box.cls[0])  # Clase detectada
        if model.names[cls_id] == 'person':
            x1, y1, x2, y2 = box.xyxy[0].tolist()  # Coordenadas de la caja
            print(f"Persona detectada en: ({x1}, {y1}), ({x2}, {y2})")

Persona detectada en: (112.66673278808594, 18.499736785888672), (423.9985656738281, 377.3861083984375)
Persona detectada en: (34.229591369628906, 84.89447021484375), (108.04861450195312, 295.09075927734375)
Persona detectada en: (537.9708251953125, 125.15662384033203), (592.1982421875, 233.28688049316406)
Persona detectada en: (371.10302734375, 67.37295532226562), (427.9504089355469, 314.15460205078125)
Persona detectada en: (117.51016998291016, 122.64320373535156), (219.3135528564453, 310.6005554199219)


In [74]:
# Mostrar la imagen con las detecciones
results[0].plot()  # Dibuja las cajas
cv2.imshow("Detecciones", results[0].plot())
cv2.waitKey(0)
cv2.destroyAllWindows()

# Video

In [75]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np
from collections import Counter

print("GPU disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Usando:", torch.cuda.get_device_name(0))

model = YOLO("yolov8n.pt")
model_line = YOLO("yolov8n_line_detail.pt")  
model.to('cuda' if torch.cuda.is_available() else 'cpu')
model_line.to('cuda' if torch.cuda.is_available() else 'cpu')
print("Modelo cargado en:", model.device)

GPU disponible: True
Usando: NVIDIA GeForce RTX 4060
Modelo cargado en: cuda:0


In [79]:
cap = cv2.VideoCapture("SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_720p.mkv")

field_map = {
    # LÍNEAS DE BORDE
    "Side line left":     (0, 0, 0, 68),
    "Side line right":    (105, 0, 105, 68),
    "Side line top":      (0, 68, 105, 68),
    "Side line bottom":   (0, 0, 105, 0),
    "Middle line":        (52.5, 0, 52.5, 68),

    # CÍRCULOS
    "Circle central":     {"center": (52.5, 34), "radius": 9.15},

    # ÁREA GRANDE IZQUIERDA
    "Big rect. left main":   (16.5, 13.85, 16.5, 54.15),
    "Big rect. left top":    (0, 54.15, 16.5, 54.15),
    "Big rect. left bottom": (0, 13.85, 16.5, 13.85),

    # ÁREA GRANDE DERECHA
    "Big rect. right main":   (88.5, 13.85, 88.5, 54.15),
    "Big rect. right top":    (105, 54.15, 88.5, 54.15),
    "Big rect. right bottom": (105, 13.85, 88.5, 13.85),

    # ÁREA CHICA IZQUIERDA
    "Small rect. left main":   (5.5, 24.84, 5.5, 43.16),
    "Small rect. left top":    (0, 43.16, 5.5, 43.16),
    "Small rect. left bottom": (0, 24.84, 5.5, 24.84),

    # ÁREA CHICA DERECHA
    "Small rect. right main":   (99.5, 24.84, 99.5, 43.16),
    "Small rect. right top":    (105, 43.16, 99.5, 43.16),
    "Small rect. right bottom": (105, 24.84, 99.5, 24.84),
}

def segment_to_line(x1, y1, x2, y2, name):
    A = y2 - y1
    B = x1 - x2
    C = x2 * y1 - x1 * y2
    return A, B, -C, name

def intersection(line1, line2):
    A1, B1, C1, NAME1 = line1
    A2, B2, C2, NAME2 = line2
    D = A1 * B2 - A2 * B1
    if D == 0:
        return None  # Son paralelas
    x = (B2 * C1 - B1 * C2) / D
    y = (A1 * C2 - A2 * C1) / D
    return x, y, NAME1, NAME2

#Lineas de campo
field_lines_map = []
for name, coords in field_map.items():
    if isinstance(coords, tuple):  # es una línea
        x1, y1, x2, y2 = coords
        field_lines_map.append(segment_to_line(x1, y1, x2, y2, name))

if not cap.isOpened():
    print("No se pudo abrir el video.")
    exit()
# Rango para el color blanco
lower_white = np.array([30, 10, 210])
upper_white = np.array([110, 70, 255])
prev_frame = None
while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    lines = model_line.predict(frame, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False)
    annotated_frame = frame.copy()
    positions = []
    index = 0
    for result in results:
        boxes = result.boxes
        detections = []
        for box in boxes:
            cls_id = int(box.cls[0])  
            if model.names[cls_id] == 'person':
                x1, y1, x2, y2 = box.xyxy[0].tolist()  
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                person_section = annotated_frame[y1:y2, x1:x2]
                small_section = cv2.resize(person_section, (20, 20), interpolation=cv2.INTER_LINEAR)
                hsv_section = cv2.cvtColor(small_section, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv_section, np.array([35, 40, 40]), np.array([85, 255, 255]))
                non_green_pixels = hsv_section[mask == 0]
                quantized_pixels = (non_green_pixels // 32) * 32
                pixels = quantized_pixels.reshape(-1, 3)
                pixels_list = [tuple(pixel) for pixel in pixels]
                if len(pixels_list) > 0:
                    most_common = Counter(pixels_list).most_common(1)[0][0]
                    domimant_color_rgb = cv2.cvtColor(np.uint8([[most_common]]), cv2.COLOR_HSV2BGR)[0][0]
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), tuple(int(c) for c in domimant_color_rgb), 2)
                    positions.append({"pos":((x1+x2)/2, max(y1, y2)),"type": model.names[cls_id], "index": index, "color": tuple(int(c) for c in domimant_color_rgb)})
                else:
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    positions.append({"pos":((x1+x2)/2, max(y1, y2)),"type": model.names[cls_id], "index": index, "color": (0, 255, 0)})
                index += 1
            #balon
            if model.names[cls_id] == 'sports ball':
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                positions.append({"pos":((x1+x2)/2, max(y1, y2)),"type": model.names[cls_id], "index": index, "color": (0, 0, 255)})
                index += 1
    # Dibuja las líneas detectadas
    field_lines = []
    for line in lines:
        boxes = line.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            if model_line.names[cls_id] in ['Circle right', 'Circle central', 'Circle left']:
                pass
            elif model_line.names[cls_id] in ["Goal right crossbar","Goal right post left","Goal right post right","Goal left post left","Goal left post left ","Goal left post left","Goal left post right","Goal left crossbar", 'unknown']:
                pass
            else:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                #cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #vamos a hacer deteccion de bordes en esta zona
                roi = annotated_frame[y1:y2, x1:x2]
                # Convertir a escala de grises
                gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                # Aplicar un filtro de suavizado
                blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)
                # Aplicar Canny para detectar bordes
                edges = cv2.Canny(blurred_roi, 50, 150)
                # mostrar los bordes
                lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=20, maxLineGap=10)

                if lines is not None:
                    angles = []
                    long_line = 0
                    longest_line = None
                    for line in lines:
                        x1_line, y1_line, x2_line, y2_line = line[0]
                        # Calcular la longitud de la línea
                        line_length = np.sqrt((x2_line - x1_line) ** 2 + (y2_line - y1_line) ** 2)
                        if line_length > long_line:
                            long_line = line_length
                            longest_line = line[0]
                        # Calcular dirección en grados
                        angle_rad = np.arctan2((y2_line - y1_line), (x2_line - x1_line))
                        angle_deg = np.degrees(angle_rad)
                        angles.append(angle_deg)
                    """ if longest_line is not None:
                        x1_line, y1_line, x2_line, y2_line = longest_line
                        cv2.line(annotated_frame, (x1 + x1_line, y1 + y1_line), (x1 + x2_line, y1 + y2_line), (0, 255, 255), 2)
                        angle_rad = np.arctan2((y2_line - y1_line), (x2_line - x1_line))
                        angle_deg = np.degrees(angle_rad)
                        if angle_deg > 0:
                            cv2.line(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                            field_lines.append({"pos":(x1, y1, x2, y2),"type": model_line.names[cls_id]})
                        else:
                            cv2.line(annotated_frame, (x1, y2), (x2, y1), (0, 255, 0), 2)
                            field_lines.append({"pos":(x1, y2, x2, y1),"type": model_line.names[cls_id]}) """
                    # Calcular el ángulo promedio
                    if len(angles) > 0:
                        avg_angle = np.mean(angles)
                        print(f"Ángulo promedio: {avg_angle:.2f} grados, {model_line.names[cls_id]}")
                        if avg_angle > 0:
                            cv2.line(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            field_lines.append({"pos":(x1, y1, x2, y2),"type": model_line.names[cls_id]})
                        else:
                            cv2.line(annotated_frame, (x1, y2), (x2, y1), (255, 0, 0), 2)
                            field_lines.append({"pos":(x1, y2, x2, y1),"type": model_line.names[cls_id]})                    
                    
    intersections = []
    names = []           
    for i, line1 in enumerate(field_lines):
        for j, line2 in enumerate(field_lines):
            if i >= j:
                continue
            if line1 != line2:
                pos1 = line1["pos"]
                pos2 = line2["pos"]
                # Convertir las coordenadas de la línea a la forma Ax + By + C = 0
                A1, B1, C1, NAME1 = segment_to_line(pos1[0], pos1[1], pos1[2], pos1[3], line1["type"])
                A2, B2, C2, NAME2 = segment_to_line(pos2[0], pos2[1], pos2[2], pos2[3], line2["type"])
                # Calcular la intersección
                intersection_point = intersection((A1, B1, C1, NAME1), (A2, B2, C2, NAME2))
                if intersection_point is not None:
                    x, y, name1, name2 = intersection_point
                    if x > 0 and y >0:
                        intersections.append({"pos":(x, y),"lines": (name1, name2)})
                        names.append((name1, name2))
                        cv2.circle(annotated_frame, (int(intersection_point[0]), int(intersection_point[1])), 5, (255, 255, 0), -1)

    # Intersección de líneas del campo        
    intersections_field = []
    for i, line1 in enumerate(field_lines_map):
        for j, line2 in enumerate(field_lines_map):
            if i >= j:
                continue
            point = intersection(line1, line2)
            if point is not None:
                x, y, name1, name2 = point
                if (name1, name2) in names or (name2, name1) in names:
                    intersections_field.append({
                        "pos": (x, y),
                        "lines": (name1, name2)
                    })        
           
    matched_pairs = []
    for i in intersections:
        for f in intersections_field:
            if set(i["lines"]) == set(f["lines"]):
                matched_pairs.append((i["pos"], f["pos"]))  # (imagen, cancha)
                break       
            
    # Homografía 2d
    positions_homography = []
    intersection_homography = []        
    if len(matched_pairs) > 4:
        
        src_pts = np.array([pt[0] for pt in matched_pairs], dtype=np.float32)
        dst_pts = np.array([pt[1] for pt in matched_pairs], dtype=np.float32)
        
        # Calcular la homografía
        H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
        print("Homografía calculada")
        # Aplicar la homografía a los puntos de la imagen
        
        for pos in positions:
            try:
                point = np.array([pos["pos"][0], pos["pos"][1], 1], dtype=np.float32)
                transformed_point = np.dot(H, point)
                transformed_point /= transformed_point[2]  # Normalizarq
                positions_homography.append({"pos":(transformed_point[0], transformed_point[1]),"type": pos["type"], "index": pos["index"], "color": pos["color"]})
            except Exception as e:
                print(f"Error al aplicar homografía a la posición {pos['index']}: {e}")
                continue
        # Aplicar la homografía a los puntos de intersección
        for pos in intersections:
            try:
                point = np.array([pos["pos"][0], pos["pos"][1], 1], dtype=np.float32)
                transformed_point = np.dot(H, point)
                transformed_point /= transformed_point[2]  # Normalizar
                intersection_homography.append({"pos":(transformed_point[0], transformed_point[1]),"lines": pos["lines"]})     
            except Exception as e:
                print(f"Error al aplicar homografía a la posición {pos['lines']}: {e}")
                continue           
    # Draw minimap of the field
    # Crear una imagen en blanco para el minimapa del campo
    field_map_img = np.ones((800, 1200, 3), dtype=np.uint8) * 255

    # Escalado para ajustar el campo a la imagen
    scale_x = 10  # 1 metro = 10 pixeles
    scale_y = 10
    offset_x = 50  # margen izquierdo
    offset_y = 50  # margen superior

    def to_img_coords(x, y):
        # Convierte coordenadas de metros a pixeles en la imagen
        return int(x * scale_x + offset_x), int(field_map_img.shape[0] - (y * scale_y + offset_y))

    # Dibujar líneas y círculos del campo
    for name, coords in field_map.items():
        if isinstance(coords, tuple):  # Línea
            x1, y1, x2, y2 = coords
            pt1 = to_img_coords(x1, y1)
            pt2 = to_img_coords(x2, y2)
            cv2.line(field_map_img, pt1, pt2, (255, 0, 0), 2)
            if name == "Middle line":
                center_x = x1
                center_y = (y1 + y2) / 2
                center_pt = to_img_coords(center_x, center_y)
                cv2.circle(field_map_img, center_pt, 5, (255, 0, 0), -1)
        elif isinstance(coords, dict):  # Círculo
            center = to_img_coords(coords["center"][0], coords["center"][1])
            radius = int(coords["radius"] * scale_x)
            cv2.circle(field_map_img, center, radius, (0, 255, 0), 2)

    # Dibujar posiciones proyectadas por homografía
    for pos in positions_homography:
        px, py = to_img_coords(pos["pos"][0], pos["pos"][1])
        color = pos["color"]
        cv2.circle(field_map_img, (px, py), 7, color, -1)
        cv2.putText(field_map_img, str(pos["index"]), (px-10, py-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.putText(field_map_img, "Minimapa del campo y posiciones proyectadas", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    # Mostrar el campo
    cv2.imshow("Campo", field_map_img)                  
                    
                    
    # Mostrar la imagen con las detecciones
    cv2.imshow("Detecciones", annotated_frame)
    # Esperar hasta que se presione una tecla para pasar al siguiente fotograma
    key = cv2.waitKey(0)
    # Si se presiona 'q', salir del bucle
    if key & 0xFF == ord('q'):
        break
    """ if cv2.waitKey(1) & 0xFF == ord('q'):
        break """
cap.release()
cv2.destroyAllWindows()

Ángulo promedio: -67.24 grados, Middle line
Ángulo promedio: -60.08 grados, Middle line
Ángulo promedio: 12.50 grados, Middle line
Ángulo promedio: -45.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -89.84 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: -89.72 grados, Middle line
Ángulo promedio: -90.00 grados, Middle line
Ángulo promedio: 0.09 grados, Side line top
Ángulo promedio: -16.01 grados, Big rect. left main
Ángulo promedio: 0.07 grados, Side line top
Ángulo promedio: 0.21 grados, Side line top
Ángulo promedio: -16.41 grados, Big rect. left main
Ángulo promedio: 0.27 grados, Side line top
Ángulo promedio: 